# **ANÁLISE E EXPLORAÇÃO DOS DADOS**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import warnings
warnings.filterwarnings("ignore")

https://www.kaggle.com/uciml/breast-cancer-wisconsin-data 

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Cursos_Udemy/otimizacao_hiperparametro/dados/data_cancer.csv',
                    sep=',', encoding='utf-8')

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df2 = df.drop(columns=['id','Unnamed: 32'])

In [ ]:
df2.head()

In [ ]:
df2.dtypes

In [ ]:
df3 = pd.DataFrame.copy(df2)

In [ ]:
df3['diagnosis'].replace({'M':0, 'B': 1}, inplace=True)
df3.head()

In [ ]:
df3.dtypes

In [ ]:
# Fração de tumores benignos (1) e malignos (0)

df3.diagnosis.value_counts() / len(df3.diagnosis)

# **PRÉ-PROCESSAMENTO**

## **Separação em Variáveis Previsoras e Alvo**

In [ ]:
previsores = df3.iloc[:, 1:32].values
previsores

In [ ]:
previsores.shape

In [ ]:
alvo = df3.iloc[:, 0].values
alvo

In [ ]:
alvo.shape

**Divisão em dados de treino e teste**

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_treino, X_teste, y_treino, y_teste = train_test_split(
    previsores, alvo, test_size=0.3, random_state=0)

X_treino.shape, X_teste.shape

# **SEM OTIMIZAR HIPERPARÂMETROS**

In [ ]:
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier

In [ ]:
# Desempenho validação cruzada
desempenho_cross_val = cross_val_score(estimator = XGBClassifier(random_state=0),
                                       X = X_treino, 
                                       y = y_treino,
                                       scoring = 'roc_auc',
                                       cv = 7)

print('Desempenho médio de AUC-ROC: {}'.format(round(desempenho_cross_val.mean(),4)))

# **GRID SEARCH**

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV

In [ ]:
from xgboost import XGBClassifier

In [ ]:
# Gerando o modelo
xgb = XGBClassifier(random_state=0)

In [ ]:
# Determinando o espaço do hiperparâmetro
param_grid = dict(
    n_estimators=[10, 50, 100, 150, 200, 250],
    learning_rate=[0.001, 0.01, 0.05, 0.1, 0.15, 0.2],
    max_depth=[1,2,3,4,None],
    objective=('binary:logistic','binary:logitraw','binary:hinge')
    )

print('Número de combinações de Hiperparâmetros: ', 
      len(param_grid['n_estimators']) * len(param_grid['learning_rate']) * len(param_grid['max_depth'])*len(param_grid['objective']))

In [ ]:
# Configurando a procura com o Grid search
grid_search = GridSearchCV(xgb, param_grid, scoring='roc_auc', cv=7)

In [ ]:
# Configurando os melhores hiperparâmetros
grid_search.fit(X_treino, y_treino)

In [ ]:
# Melhores Hiperparâmetros
grid_search.best_params_

In [ ]:
print('Score médio do melhor modelo: {}'.format(round(grid_search.best_score_,4)))

In [ ]:
# Contagem do tempo de execução do treinamento
%timeit -n 1 -r 1 grid_search.fit(X_treino, y_treino)
print('Tempo para realizar a otimização:') 

In [ ]:
# Relação dos dados para todos os modelos

results = pd.DataFrame(grid_search.cv_results_)
print(results.shape)
results.head(3)

In [ ]:
# Ordenando os melhores resultados
results.sort_values(by='mean_test_score', ascending=False, inplace=True)

results.reset_index(drop=True, inplace=True)

results[['param_n_estimators', 'param_learning_rate', 'param_max_depth', 'param_objective',
    'mean_test_score', 'std_test_score']].head()

In [ ]:
# Gráfico do desempenho do modelo

results['mean_test_score'].plot(yerr=[results['std_test_score'], results['std_test_score']], subplots=True)

plt.ylabel('Score médio de teste')
plt.xlabel('Combinação de Hyperparâmetros');

In [ ]:
X_treino_prev = grid_search.predict_proba(X_treino)[:,1]
X_teste_prev = grid_search.predict_proba(X_teste)[:,1]

print('Roc_Auc de Treino: ', roc_auc_score(y_treino, X_treino_prev))
print('Roc_Auc de Teste: ', roc_auc_score(y_teste, X_teste_prev))

In [ ]:
# Função para avaliar o desempenho do modelo com base em hiperparâmetros únicos

def summarize_by_param(hparam):
    
    tmp = pd.concat([
        results.groupby(hparam)['mean_test_score'].mean(),
        results.groupby(hparam)['mean_test_score'].std(),
    ], axis=1)

    tmp.columns = ['mean_test_score', 'std_test_score']
    
    return tmp

In [ ]:
# Desempenho de n_estimators

tmp = summarize_by_param('param_n_estimators')

tmp.head(6)

In [ ]:
tmp['mean_test_score'].plot(yerr=[tmp['std_test_score'], tmp['std_test_score']], subplots=True)
plt.ylabel('roc-auc');

O hiperparâmetro ideal está em algum lugar acima de 100 até 250 podendo testar acima de 250.

In [ ]:
# Desempenho de max_depth

tmp = summarize_by_param('param_max_depth')
tmp.head(6)

In [ ]:
tmp = summarize_by_param('param_max_depth')
tmp['mean_test_score'].plot(yerr=[tmp['std_test_score'], tmp['std_test_score']], subplots=True)
plt.ylabel('roc-auc');

O hiperparâmetro ideal parece estar em algum lugar entre 2 e 3



In [ ]:
# Desempenho de min_samples_split

tmp = summarize_by_param('param_learning_rate')

tmp.head(6)

In [ ]:
tmp = summarize_by_param('param_learning_rate')
tmp['mean_test_score'].plot(yerr=[tmp['std_test_score'], tmp['std_test_score']], subplots=True);

O hiperparâmetro ideal parece estar em algum lugar entre 0.1 e 0.2, podendo testar acima de 0.2.

In [ ]:
# Determinação do novo espaço do hiperparâmetro
param_grid2 = dict(
    n_estimators=[100, 125, 150, 175, 200, 225, 250, 275],
    max_depth=[2,2.25, 2.5, 2.75,3],
    learning_rate= [0.1, 0.12, 0.14, 0.16, 0.18, 0.2, 0.22, 0.24],
    )

# Configurando O Grid Search
grid_search2 = GridSearchCV(xgb, param_grid2, scoring='roc_auc', cv=7)

# Encontrando o melhor parâmetro
grid_search2.fit(X_treino, y_treino)

In [ ]:
# Melhores Hiperparâmetros

grid_search2.best_params_

In [ ]:
results = pd.DataFrame(grid_search2.cv_results_)

results.sort_values(by='mean_test_score', ascending=False, inplace=True)

results.reset_index(drop=True, inplace=True)

results[['param_n_estimators', 'param_max_depth', 'param_learning_rate',
    'mean_test_score', 'std_test_score']].head()

In [ ]:
X_treino_prev = grid_search2.predict_proba(X_treino)[:,1]
X_teste_prev = grid_search2.predict_proba(X_teste)[:,1]

print('ROC-AUC Treino: ', roc_auc_score(y_treino, X_treino_prev))
print('ROC-AUC Teste: ', roc_auc_score(y_teste, X_teste_prev))

In [ ]:
print('Score médio do melhor modelo: {}'.format(round(grid_search.best_score_,4)))

In [ ]:
print('Score médio do melhor modelo: {}'.format(round(grid_search2.best_score_,4)))

# **RANDOM SEARCH**

In [ ]:
from scipy import stats

In [ ]:
stats.randint.rvs(1, 5)

In [ ]:
stats.uniform.rvs(0, 1)

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
# Gerando o modelo
xgb = XGBClassifier(random_state=0)

In [ ]:
# Determinação do espaço de hiperparâmetros
param_random = dict(
    n_estimators=stats.randint(10, 300),
    learning_rate=stats.uniform(0.001, 0.3),
    max_depth=stats.randint(1, 8),
    objective=('binary:logistic','binary:logitraw','binary:hinge')
    )

In [ ]:
# Configurando a procura com o Random Search
random_search = RandomizedSearchCV(xgb,
                            param_random,
                            scoring='roc_auc',
                            cv=7,
                            n_iter = 100,
                            random_state=0,
                            n_jobs=4)

In [ ]:
# Analisando os hiperparâmetros
random_search.fit(X_treino, y_treino)

In [ ]:
# Melhores hiperparâmetros
random_search.best_params_

In [ ]:
# Resultados para todos os modelos

results = pd.DataFrame(random_search.cv_results_)

print(results.shape)

results.head()

In [ ]:
# Ordenando os modelos com melhores desempenhos
results.sort_values(by='mean_test_score', ascending=False, inplace=True)

results.reset_index(drop=True, inplace=True)

results[['param_n_estimators', 'param_learning_rate', 'param_max_depth', 'param_objective',
    'mean_test_score', 'std_test_score']].head()

In [ ]:
# Gráfico dos modelos conforme desempenho

results['mean_test_score'].plot(yerr=[results['std_test_score'], results['std_test_score']], subplots=True)

plt.ylabel('Score médio')
plt.xlabel('Combinação de hiperparâmetros');

In [ ]:
X_treino_prev = random_search.predict_proba(X_treino)[:,1]
X_teste_prev = random_search.predict_proba(X_teste)[:,1]

print('Train roc_auc: ', roc_auc_score(y_treino, X_treino_prev))
print('Test roc_auc: ', roc_auc_score(y_teste, X_teste_prev))

In [ ]:
print('Score médio do melhor modelo: {}'.format(round(random_search.best_score_,4)))

# **OTIMIZAÇÃO BAYESIANA**

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
pip install scikit-optimize

In [ ]:
# Importando o Bayes Search com cross-validation:
from skopt import BayesSearchCV

In [ ]:
from xgboost import XGBClassifier

In [ ]:
# Importando espaços de busca
from skopt.space import Integer, Real, Categorical

In [ ]:
# Definindo a otimização Bayesiana:
bayes_search = BayesSearchCV(estimator = XGBClassifier(random_state = 0),
                              search_spaces = {'n_estimators':Integer(1,300),
                                               'learning_rate': Real(0.001,0.3),
                                               'max_depth':Integer(1,8),
                                               'objective': Categorical(['binary:logistic','binary:logitraw','binary:hinge'])},
                              scoring='roc_auc', 
                              cv = 7,
                              n_iter = 50)

In [ ]:
# Realizando a otimização Bayesiana
bayes_search.fit(X_treino, y_treino)

In [ ]:
#Vamos ver informações relevantes:
print('Melhor número de estimadores: {}'.format(bayes_search.best_params_['n_estimators']))
print('Melhor taxa de aprendizagem: {}'.format(bayes_search.best_params_['learning_rate']))
print('Melhor profundidade maxima: {}'.format(bayes_search.best_params_['max_depth']))
print('Melhor estimador de perda: {}'.format(bayes_search.best_params_['objective']))
print('Score médio do melhor modelo: {}'.format(round(bayes_search.best_score_,4)))

In [ ]:
X_treino_prev = bayes_search.predict_proba(X_treino)[:,1]
X_teste_prev = bayes_search.predict_proba(X_teste)[:,1]

print('Train roc_auc: ', roc_auc_score(y_treino, X_treino_prev))
print('Test roc_auc: ', roc_auc_score(y_teste, X_teste_prev))